# Extract Data from Schools

In [1]:
# Global Packages
import requests
import certifi
from time import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


### Parameters

In [2]:
BASE_URL = "https://www.redacademica.edu.co/colegios?name=&field_localidad_target_id=All"
HEADERS = { "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
           "Host": "www.redacademica.edu.co" }

In [20]:
# find all a's with href
def get_urls_dict(soup):
    """
    Getting school URLS from the given soup object
    """
    urls_dict = {}
    a_objects = soup.find_all("a", href=True)
    for a_object in a_objects:
        if "colegios/" in str.lower(a_object["href"]):
            new_url = "https://www.redacademica.edu.co" + a_object["href"]
            urls_dict[a_object.text] = new_url

    if "\n\n\n\n" in urls_dict:
        del urls_dict["\n\n\n\n"]

### Playground

In [3]:
r = requests.get(BASE_URL, headers=HEADERS)
soup = BeautifulSoup(r.content, "html.parser")

In [19]:
len(urls_dict)

20